## 1. Setup

In [ ]:
%%bash
pip install .. -q
pip install ipywidgets -q

In [7]:
import ipywidgets as widgets
import torch
from IPython.display import Markdown, display

import henets

# 2. Training and testing visual models for classification

In [ ]:
# widget parameters
MODEL_OPTIONS = ["lenet"]
DATASET_OPTIONS = ["mnist"]
PIPELINE_STEPS_PATH_OPTIONS = {
    "lenet": [
        "../pipeline_steps/vision_models_for_classification/lenet/without_approximations.json",
        "../pipeline_steps/vision_models_for_classification/lenet/pipeline_steps_Q.json",
        "../pipeline_steps/vision_models_for_classification/lenet/pipeline_steps_TQ1.json",
        "../pipeline_steps/vision_models_for_classification/lenet/pipeline_steps_AVGpool.json",
        "../pipeline_steps/vision_models_for_classification/lenet/pipeline_steps_AVGpool_Q.json",
        "../pipeline_steps/vision_models_for_classification/lenet/pipeline_steps_AVGpool_TQ1.json",
    ],
}
ACCELERATOR_OPTIONS = ["cpu", "gpu"]
STARTING_BATCH_SIZE = 32
MIN_BATCH_SIZE = 1
MAX_BATCH_SIZE = 256
STARTING_EXPERIMENT_NAME = f"{MODEL_OPTIONS[0]}_{DATASET_OPTIONS[0]}"
STARTING_ADDITIONAL_ARGUMENTS = ""

# cli command parameters
model = MODEL_OPTIONS[0]
dataset = DATASET_OPTIONS[0]
pipeline_steps_path = PIPELINE_STEPS_PATH_OPTIONS[MODEL_OPTIONS[0]][0]
experiment_name = STARTING_EXPERIMENT_NAME
accelerator = (
    ACCELERATOR_OPTIONS[1] if torch.cuda.is_available() else ACCELERATOR_OPTIONS[0]
)
batch_size = STARTING_BATCH_SIZE
additional_arguments = STARTING_ADDITIONAL_ARGUMENTS

# Dropdown widget for the the model selection
#############################################

# creating a dropdown widget for the model selection
model_dropdown = widgets.Dropdown(
    options=MODEL_OPTIONS,
    value="lenet",
    description="Model:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the selected model


def update_model(change):
    global model
    model = change.new


# observing the dropdown widget for value changes
model_dropdown.observe(update_model, names="value")

# Dropdown widget for the the dataset selection
###############################################

# creating a dropdown widget for the dataset selection
dataset_dropdown = widgets.Dropdown(
    options=DATASET_OPTIONS,
    value=DATASET_OPTIONS[0],
    description="Dataset:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the selected dataset


def update_dataset(change):
    global dataset, experiment_name
    dataset = change.new
    experiment_name = f"{model_dropdown.value}_{dataset}"


# observing the dropdown widget for value changes
dataset_dropdown.observe(update_dataset, names="value")

# Slider widget for the batch_size selection
############################################

# creating an slider widget for the batch_size selection
batch_size_slider = widgets.IntSlider(
    value=STARTING_BATCH_SIZE,
    min=MIN_BATCH_SIZE,
    max=MAX_BATCH_SIZE,
    step=1,
    description="Batch Size:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)
batch_size_slider.style.handle_color = "lightblue"

# defining a function to update the variable with the selected batch_size


def update_batch_size(change):
    global batch_size
    batch_size = change.new


# observing the slider widget for value changes
batch_size_slider.observe(update_batch_size, names="value")

# Text widget for the additional arguments
##########################################

additional_arguments_text = widgets.Text(
    value=STARTING_ADDITIONAL_ARGUMENTS,
    placeholder="--additional-arg arg_value ...",
    description="Additional arguments:\n",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the specified additional_arguments


def update_additional_arguments(change):
    global additional_arguments
    additional_arguments = change.new


# observing the dropdown widget for value changes
additional_arguments_text.observe(update_additional_arguments, names="value")

# Dropdown widget for the the pipeline steps path selection
###########################################################

# creating a dropdown widget for the pipeline_steps_path selection
pipeline_steps_path_dropdown = widgets.Dropdown(
    options=PIPELINE_STEPS_PATH_OPTIONS[MODEL_OPTIONS[0]],
    value=PIPELINE_STEPS_PATH_OPTIONS[MODEL_OPTIONS[0]][0],
    description="Select pipeline steps file path:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the selected pipeline_steps_path


def update_pipeline_steps_path(change):
    global pipeline_steps_path
    pipeline_steps_path = change.new


# observing the dropdown widget for value changes
pipeline_steps_path_dropdown.observe(update_pipeline_steps_path, names="value")

# function to update options of pipeline_steps_path_dropdown based on the value of model_dropdown


def update_pipeline_steps_path_dropdown_options(change):
    for option in PIPELINE_STEPS_PATH_OPTIONS.keys():
        if change.new == option:
            pipeline_steps_path_dropdown.options = PIPELINE_STEPS_PATH_OPTIONS[option]
            break


# observing the dropdown widget for value changes
model_dropdown.observe(update_pipeline_steps_path_dropdown_options, "value")

# Text widget for the the experiment name selection
###################################################

# creating a text widget for the experiment name selection
experiment_name_text = widgets.Text(
    value=STARTING_EXPERIMENT_NAME,
    placeholder="Enter experiment name",
    description="Experiment name:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the specified experiment name


def update_experiment_name(change):
    global experiment_name
    experiment_name = change.new


# observing the text widget for value changes
experiment_name_text.observe(update_experiment_name, names="value")

# function to update the experiment name based on the value of model_dropdown, dataset_dropdown and pipeline_steps_path_dropdown


def update_experiment_name_text(change):
    approximations = pipeline_steps_path_dropdown.value.split("/")[-1].split(".")[0]
    if approximations == "without_approximations":
        experiment_name_text.value = f"{model_dropdown.value}_{dataset_dropdown.value}"
    else:
        approximations = approximations.replace("pipeline_steps_", "")
        experiment_name_text.value = (
            f"{model_dropdown.value}_{dataset_dropdown.value}_{approximations}"
        )


# observing the model selection dropdown widget for value changes
model_dropdown.observe(update_experiment_name_text, "value")
# observing the dataset selection dropdown widget for value changes
dataset_dropdown.observe(update_experiment_name_text, "value")
# observing the pipeline_steps_path selection dropdown widget for value changes
pipeline_steps_path_dropdown.observe(update_experiment_name_text, "value")

# Dropdown widget for the the accelerator selection
###################################################

# creating a dropdown widget for the accelerator selection
accelerator_dropdown = widgets.Dropdown(
    options=(
        ACCELERATOR_OPTIONS if torch.cuda.is_available() else [ACCELERATOR_OPTIONS[0]]
    ),
    value=(
        ACCELERATOR_OPTIONS[1] if torch.cuda.is_available() else ACCELERATOR_OPTIONS[0]
    ),
    description="Accelerator:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# defining a function to update the variable with the selected accelerator


def update_accelerator(change):
    global accelerator
    accelerator = change.new


# observing the dropdown widget for value changes
accelerator_dropdown.observe(update_accelerator, names="value")

# Reset button
##############

# creating a reset button
reset_button = widgets.Button(
    description="Reset values",
    button_style="primary",
    layout=widgets.Layout(width="99.5%", height="30px"),
)

# function to reset all the widgets values


def reset_dropdown(event):
    model_dropdown.value = MODEL_OPTIONS[0]
    dataset_dropdown.value = DATASET_OPTIONS[0]
    accelerator_dropdown.value = (
        ACCELERATOR_OPTIONS[1] if torch.cuda.is_available() else ACCELERATOR_OPTIONS[0]
    )
    batch_size_slider.value = STARTING_BATCH_SIZE
    pipeline_steps_path_dropdown.value = PIPELINE_STEPS_PATH_OPTIONS[MODEL_OPTIONS[0]][
        0
    ]
    experiment_name_text.value = STARTING_EXPERIMENT_NAME
    additional_arguments_text.value = STARTING_ADDITIONAL_ARGUMENTS


# linking the reset button to reset function
reset_button.on_click(reset_dropdown)

# Output widget to display the approximations that will be used in the experiment
#################################################################################

# output widget to display Markdown content
output = widgets.Output()

# function to display Markdown content based on the selected pipeline_steps_path


def display_markdown(change):
    selected_option = (change.new).split("/")[-1].split(".")[0]
    if selected_option == "without_approximations":
        markdown_content = "## Model trained and tested without approximations."
    else:
        markdown_content = "## Model trained and tested the following approximations:\n"
        selected_option = selected_option.replace("pipeline_steps_", "").split("_")
        if "TQ1" in selected_option:
            markdown_content = (
                markdown_content + "*    ReLU → **Trainable Quadratic activation**\n"
            )
        if "Q" in selected_option:
            markdown_content = (
                markdown_content + "*    ReLU → **Quadratic activation**\n"
            )
        if "BN" in selected_option:
            markdown_content = (
                markdown_content
                + "*    Layer Normalization → **Batch Normalization**\n"
            )
        if "AVGpool" in selected_option:
            markdown_content = (
                markdown_content + "*    Max Pooling → **Average Pooling**\n"
            )

    with output:
        output.clear_output()
        display(Markdown(markdown_content))


# observing the dropdown widget for value changes
pipeline_steps_path_dropdown.observe(display_markdown, "value")

# displaying the widgets
vbox = widgets.VBox(
    [
        model_dropdown,
        dataset_dropdown,
        batch_size_slider,
        additional_arguments_text,
        pipeline_steps_path_dropdown,
        experiment_name_text,
        accelerator_dropdown,
        reset_button,
        output,
    ]
)
centered_vbox = widgets.Box(
    children=[vbox],
    layout=widgets.widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
    ),
)
display(centered_vbox)
with output:
    display(Markdown("## Model trained and tested without approximations."))

In [ ]:
!vision-model-classification-pipeline --model $model\
                                      --dataset_name $dataset\
                                      --batch_size $batch_size\
                                      --pin_memory\
                                      --persistent_workers\
                                      --accelerator $accelerator\
                                      --experiment_name $experiment_name\
                                      --pipeline_steps_path $pipeline_steps_path\
                                      $additional_arguments